In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
import pandas as pd
from env import host, username, password
from sklearn.model_selection import train_test_split


import wrangle_project
import warnings
warnings.filterwarnings("ignore")

In [3]:

def get_telco_data():
    """
    This function connects to the telco_churn database and retrieves data from the customers, contract_types,
    internet_service_types, and payment_types tables. The resulting DataFrame contains all columns from these
    tables and is returned by the function.
    """
   
    # create the connection url
    url = f'mysql+pymysql://{username}:{password}@{host}/telco_churn'

    # read the SQL query into a DataFrame
    query = '''
            SELECT *
            FROM customers
            JOIN contract_types USING(contract_type_id)
            JOIN internet_service_types USING(internet_service_type_id)
            JOIN payment_types USING(payment_type_id)
            '''
    df = pd.read_sql(query, url)

    return df


In [4]:
get_telco_data().head()

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,2,1,2,0002-ORFBO,Female,0,Yes,Yes,9,Yes,...,Yes,Yes,No,Yes,65.6,593.3,No,One year,DSL,Mailed check
1,2,1,1,0003-MKNFE,Male,0,No,No,9,Yes,...,No,No,Yes,No,59.9,542.4,No,Month-to-month,DSL,Mailed check
2,1,2,1,0004-TLHLJ,Male,0,No,No,4,Yes,...,No,No,No,Yes,73.9,280.85,Yes,Month-to-month,Fiber optic,Electronic check
3,1,2,1,0011-IGKFF,Male,1,Yes,No,13,Yes,...,No,Yes,Yes,Yes,98.0,1237.85,Yes,Month-to-month,Fiber optic,Electronic check
4,2,2,1,0013-EXCHZ,Female,1,Yes,No,3,Yes,...,Yes,Yes,No,Yes,83.9,267.4,Yes,Month-to-month,Fiber optic,Mailed check


In [5]:
df = get_telco_data()

In [6]:
df.head()

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,2,1,2,0002-ORFBO,Female,0,Yes,Yes,9,Yes,...,Yes,Yes,No,Yes,65.6,593.3,No,One year,DSL,Mailed check
1,2,1,1,0003-MKNFE,Male,0,No,No,9,Yes,...,No,No,Yes,No,59.9,542.4,No,Month-to-month,DSL,Mailed check
2,1,2,1,0004-TLHLJ,Male,0,No,No,4,Yes,...,No,No,No,Yes,73.9,280.85,Yes,Month-to-month,Fiber optic,Electronic check
3,1,2,1,0011-IGKFF,Male,1,Yes,No,13,Yes,...,No,Yes,Yes,Yes,98.0,1237.85,Yes,Month-to-month,Fiber optic,Electronic check
4,2,2,1,0013-EXCHZ,Female,1,Yes,No,3,Yes,...,Yes,Yes,No,Yes,83.9,267.4,Yes,Month-to-month,Fiber optic,Mailed check


In [7]:

def prep_telco(df):
    """
    This function prepares a telco DataFrame for machine learning modeling by performing the following steps:
    - Drops duplicate rows
    - Replaces blank values with NaN
    - Replaces "No internet service" with "No" for relevant columns
    - Replaces binary columns with 1 (Yes) and 0 (No)
    - Converts total_charges to a numeric data type
    - Encodes categorical columns using one-hot encoding
    - Drops original categorical columns

    Parameters:
    df (pandas DataFrame): The telco DataFrame to be prepared.

    Returns:
    pandas DataFrame: The prepared telco DataFrame.
    """
    
    # Drop duplicate rows
    df = df.drop_duplicates()

    # Replace blank values with NaN
    df = df.replace("", pd.np.nan)

    # Replace "No internet service" with "No" for relevant columns
    cols = ['online_security', 'online_backup', 'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies']
    for col in cols:
        df[col] = df[col].replace("No internet service", "No")

    # Replace binary columns with 1 (Yes) and 0 (No)
    binary_cols = ['partner', 'dependents', 'phone_service', 'multiple_lines', 'online_security', 'online_backup', 'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies', 'paperless_billing', 'churn']
    for col in binary_cols:
        df[col] = df[col].replace({"Yes": 1, "No": 0})

    # Convert total_charges to a numeric data type
    df['total_charges'] = pd.to_numeric(df['total_charges'], errors='coerce')
    
    # Encode the categorical columns
    cat_cols = ['gender', 'partner', 'dependents', 'phone_service', 'multiple_lines',
            'online_security', 'online_backup', 'device_protection', 'tech_support',
            'streaming_tv', 'streaming_movies', 'paperless_billing',
            'contract_type', 'internet_service_type', 'payment_type']

    dummy_df = pd.get_dummies(df[cat_cols], drop_first=True)

    df = pd.concat([df, dummy_df], axis=1)

    df.drop(cat_cols, axis=1, inplace=True)

    return df


In [8]:
prep_telco = prep_telco(df)

In [9]:
prep_telco.head()

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,senior_citizen,tenure,monthly_charges,total_charges,churn,gender_Male,multiple_lines_1,multiple_lines_No phone service,contract_type_One year,contract_type_Two year,internet_service_type_Fiber optic,internet_service_type_None,payment_type_Credit card (automatic),payment_type_Electronic check,payment_type_Mailed check
0,2,1,2,0002-ORFBO,0,9,65.6,593.30,0,0,0,0,1,0,0,0,0,0,1
1,2,1,1,0003-MKNFE,0,9,59.9,542.40,0,1,1,0,0,0,0,0,0,0,1
2,1,2,1,0004-TLHLJ,0,4,73.9,280.85,1,1,0,0,0,0,1,0,0,1,0
3,1,2,1,0011-IGKFF,1,13,98.0,1237.85,1,1,0,0,0,0,1,0,0,1,0
4,2,2,1,0013-EXCHZ,1,3,83.9,267.40,1,0,0,0,0,0,1,0,0,0,1


In [10]:
def train_validate_test_split(prep_telco, target, seed=123):
    '''
    This function takes in a dataframe, the name of the target variable
    (for stratification purposes), and an integer for a setting a seed
    and splits the data into train, validate and test. 
    Test is 20% of the original dataset, validate is .30*.80= 24% of the 
    original dataset, and train is .70*.80= 56% of the original dataset. 
    The function returns, in this order, train, validate and test dataframes. 
    '''
    train_validate, test = train_test_split(prep_telco, test_size=0.2, 
                                            random_state=seed, 
                                            stratify=prep_telco[target])
    train, validate = train_test_split(train_validate, test_size=0.3, 
                                       random_state=seed,
                                       stratify=train_validate[target])
    return train, validate, test

In [11]:
train, validate, test = train_validate_test_split(prep_telco, 'churn')

In [12]:
train.head()

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,senior_citizen,tenure,monthly_charges,total_charges,churn,gender_Male,multiple_lines_1,multiple_lines_No phone service,contract_type_One year,contract_type_Two year,internet_service_type_Fiber optic,internet_service_type_None,payment_type_Credit card (automatic),payment_type_Electronic check,payment_type_Mailed check
5310,3,1,3,7503-MIOGA,1,72,89.85,6697.35,0,0,1,0,0,1,0,0,0,0,0
3790,4,1,3,5329-KRDTM,1,72,77.35,5396.25,0,1,1,0,0,1,0,0,1,0,0
4398,1,2,2,6199-IWKGC,1,46,100.25,4753.85,0,0,1,0,1,0,1,0,0,1,0
2635,1,1,1,3748-FVMZZ,0,4,40.05,162.45,0,1,0,1,0,0,0,0,0,1,0
2986,2,1,1,4280-DLSHD,0,8,54.75,445.85,0,1,0,0,0,0,0,0,0,0,1


In [13]:
train, validate, test = train_validate_test_split(prep_telco, target='churn')
train.shape, validate.shape, test.shape

((3943, 19), (1691, 19), (1409, 19))

In [14]:

import pandas as pd
from scipy.stats import chi2_contingency



# Define the contingency table
cont_table = pd.crosstab(df['churn'], pd.cut(df['monthly_charges'], bins=[0, 30, 60, 100]))

# Run the chi-square test
stat, p, dof, expected = chi2_contingency(cont_table)

# Print the results
print("Chi-square test results:")
print("Test statistic:", stat)
print("p-value:", p)

if p < 0.05:
    print("Since the p-value is less than 0.05, we reject the null hypothesis and conclude that there is a significant relationship between monthly charges and churn.")
else:
    print("Since the p-value is greater than 0.05, we fail to reject the null hypothesis and conclude that there is no significant relationship between monthly charges and churn.")


Chi-square test results:
Test statistic: 356.14800634218
p-value: 4.607263511773577e-78
Since the p-value is less than 0.05, we reject the null hypothesis and conclude that there is a significant relationship between monthly charges and churn.


In [15]:


from scipy.stats import ttest_ind



# Define the two groups based on tech_support
tech_support = df[df['tech_support'] == 1]
no_tech_support = df[df['tech_support'] == 0]

# Run the t-test
stat, p = ttest_ind(tech_support['tenure'], no_tech_support['tenure'])

# Print the results
print("T-test results:")
print("Test statistic:", stat)
print("p-value:", p)

if p < 0.05:
    print("Since the p-value is less than 0.05, we reject the null hypothesis and conclude that there is a significant difference between the mean tenure of customers with and without tech support.")
else:
    print("Since the p-value is greater than 0.05, we fail to reject the null hypothesis and conclude that there is no significant difference between the mean tenure of customers with and without tech support.")


T-test results:
Test statistic: nan
p-value: nan
Since the p-value is greater than 0.05, we fail to reject the null hypothesis and conclude that there is no significant difference between the mean tenure of customers with and without tech support.


In [16]:

import numpy as np
import pandas as pd

def calculate_baseline(df, target_col):
    # Determine the most prevalent class in the target column
    mode = df[target_col].mode()[0]

    # Make all predictions using the most prevalent class
    baseline_preds = np.full(df.shape[0], mode)

    # Calculate the accuracy of the baseline model
    baseline_accuracy = (baseline_preds == df[target_col]).mean()

    # Print the baseline prediction and accuracy
    print(f"Baseline Prediction: {mode}")
    print(f"Baseline Accuracy: {baseline_accuracy:.4f}")


In [17]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Select the features and target variable
X = prep_telco.drop(columns=['churn'])
y = prep_telco['churn']

# One-hot encode categorical variables
X = pd.get_dummies(X)

# Split the dataset into training and testing subsets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Impute missing values with median
imp = SimpleImputer(strategy='median')
X_train = imp.fit_transform(X_train)
X_test = imp.transform(X_test)

# Initialize the decision tree classifier
dt = DecisionTreeClassifier(random_state=42)

# Fit the classifier to the training sample
dt.fit(X_train, y_train)

# Make predictions on the training and testing samples
y_pred_train = dt.predict(X_train)
y_pred_test = dt.predict(X_test)

# Evaluate the accuracy of the classifier on the training and testing samples
accuracy_train = dt.score(X_train, y_train)
accuracy_test = dt.score(X_test, y_test)

print("Accuracy on training sample:", accuracy_train)
print("Accuracy on testing sample:", accuracy_test)

# Evaluate the confusion matrix and classification report on the testing sample
print("Confusion matrix on testing sample:\n", confusion_matrix(y_test, y_pred_test))
print("Classification report on testing sample:\n", classification_report(y_test, y_pred_test)) 

Accuracy on training sample: 1.0
Accuracy on testing sample: 0.7757274662881476
Confusion matrix on testing sample:
 [[912 124]
 [192 181]]
Classification report on testing sample:
               precision    recall  f1-score   support

           0       0.83      0.88      0.85      1036
           1       0.59      0.49      0.53       373

    accuracy                           0.78      1409
   macro avg       0.71      0.68      0.69      1409
weighted avg       0.76      0.78      0.77      1409



In [22]:

import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate_models(train_data, val_data, test_data, feature_cols, target_col):
    # Split the data into feature and target columns
    X_train, y_train = train_data[feature_cols], train_data[target_col]
    X_val, y_val = val_data[feature_cols], val_data[target_col]
    X_test, y_test = test_data[feature_cols], test_data[target_col]

    # Create four different models
    models = [
        ('Logistic Regression', LogisticRegression(random_state=42)),
        ('Decision Tree', DecisionTreeClassifier(random_state=42)),
        ('Random Forest', RandomForestClassifier(random_state=42)),
        ('Support Vector Machine', SVC(random_state=42))
    ]

    # Train and evaluate each model on the training data
    train_scores = []
    for name, model in models:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_train)
        
        accuracy = accuracy_score(y_train, y_pred)
        precision = precision_score(y_train, y_pred)
        recall = recall_score(y_train, y_pred)
        f1 = f1_score(y_train, y_pred)

        train_scores.append((name, accuracy, precision, recall, f1))
        
    # Evaluate each model on the validation data
    val_scores = []
    for name, model in models:
        y_pred = model.predict(X_val)

        accuracy = accuracy_score(y_val, y_pred)
        precision = precision_score(y_val, y_pred)
        recall = recall_score(y_val, y_pred)
        f1 = f1_score(y_val, y_pred)

        val_scores.append((name, accuracy, precision, recall, f1))
        
    # Evaluate each model on the testing data
    test_scores = []
    for name, model in models:
        y_pred = model.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        test_scores.append((name, accuracy, precision, recall, f1))

    # Print the scores for each model and dataset
    for scores, dataset in zip([train_scores, val_scores, test_scores], ['Train', 'Validation', 'Test']):
        print(f"{dataset} Metrics:")
        for name, accuracy, precision, recall, f1 in scores:
            print(f"{name}:")
            print(f"\tAccuracy: {accuracy:.4f}")
            print(f"\tPrecision: {precision:.4f}")
            print(f"\tRecall: {recall:.4f}")
            print(f"\tF1-Score: {f1:.4f}")
    


# Define the feature and target columns
feature_cols = ['monthly_charges']
target_col = 'churn'

# Split the data into train, validation, and test sets
train_data, test_data = train_test_split(prep_telco, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.25, random_state=42)

# Evaluate the models on the train, validation, and test sets
evaluate_models(train_data, val_data, test_data, feature_cols, target_col)


Train Metrics:
Logistic Regression:
	Accuracy: 0.7375
	Precision: 0.0000
	Recall: 0.0000
	F1-Score: 0.0000
Decision Tree:
	Accuracy: 0.8225
	Precision: 0.8037
	Recall: 0.4283
	F1-Score: 0.5588
Random Forest:
	Accuracy: 0.8225
	Precision: 0.7409
	Recall: 0.4977
	F1-Score: 0.5955
Support Vector Machine:
	Accuracy: 0.7375
	Precision: 0.0000
	Recall: 0.0000
	F1-Score: 0.0000
Validation Metrics:
Logistic Regression:
	Accuracy: 0.7253
	Precision: 0.0000
	Recall: 0.0000
	F1-Score: 0.0000
Decision Tree:
	Accuracy: 0.6757
	Precision: 0.3349
	Recall: 0.1835
	F1-Score: 0.2371
Random Forest:
	Accuracy: 0.6678
	Precision: 0.3483
	Recall: 0.2403
	F1-Score: 0.2844
Support Vector Machine:
	Accuracy: 0.7253
	Precision: 0.0000
	Recall: 0.0000
	F1-Score: 0.0000
Test Metrics:
Logistic Regression:
	Accuracy: 0.7353
	Precision: 0.0000
	Recall: 0.0000
	F1-Score: 0.0000
Decision Tree:
	Accuracy: 0.6899
	Precision: 0.3621
	Recall: 0.2252
	F1-Score: 0.2777
Random Forest:
	Accuracy: 0.6785
	Precision: 0.3601
	Re

In [31]:

import xgboost as xgb

# Define the feature and target columns
feature_cols = ['monthly_charges']
target_col = 'churn'

# Split the data into train, validation, and test sets
train_data, test_data = wrangle_project.train_test_split(prep_telco, test_size=0.2, random_state=42)
train_data, val_data = wrangle_project.train_test_split(train_data, test_size=0.25, random_state=42)

# Convert the data to DMatrix format for XGBoost
dtrain = xgb.DMatrix(train_data[feature_cols], label=train_data[target_col])
dval = xgb.DMatrix(val_data[feature_cols], label=val_data[target_col])
dtest = xgb.DMatrix(test_data[feature_cols], label=test_data[target_col])

# Define the XGBoost parameters
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'eta': 0.1,
    'max_depth': 3,
    'min_child_weight': 5,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'seed': 42
}

# Train the XGBoost model
xgb_model = xgb.train(params, dtrain, num_boost_round=1000, evals=[(dval, 'Validation')], early_stopping_rounds=50, verbose_eval=False)

# Evaluate the XGBoost model on the test set
y_pred = xgb_model.predict(dtest)
y_pred_binary = [1 if y >= 0.5 else 0 for y in y_pred]

# Print the test set metrics
accuracy = accuracy_score(test_data[target_col], y_pred_binary)
precision = precision_score(test_data[target_col], y_pred_binary)
recall = recall_score(test_data[target_col], y_pred_binary)
f1 = f1_score(test_data[target_col], y_pred_binary)

print(f'Test Accuracy: {accuracy:.4f}')
print(f'Test Precision: {precision:.4f}')
print(f'Test Recall: {recall:.4f}')
print(f'Test F1-Score: {f1:.4f}')


Test Accuracy: 0.7310
Test Precision: 0.4062
Test Recall: 0.0349
Test F1-Score: 0.0642


In [29]:


# Use the XGBoost model to make predictions on the test set
test_data['predicted_churn'] = y_pred_binary

# Create a dataframe with the required columns
output_df = test_data[['customer_id', 'predicted_churn']]
output_df.columns = ['customerID', 'Churn']

# Write the dataframe to a CSV file
output_df.to_csv('churn_predictions.csv', index=False)




In [33]:
output_df.head()

,customerID,Churn
185,0301-KOBTQ,0
2715,3865-YIOTT,0
3825,5377-NDTOU,0
1807,2637-FKFSY,0
132,0224-NIJLP,0
